# Обучение модели классификации комментариев

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Требуется инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Нужна модель для классификации комментариев на позитивные и негативные. Предоставлен набор данных с разметкой о токсичности правок.Значение метрики F1 должно быть не меньше 0.75.

**Описание данных**

Данные находятся в файле *toxic_comments.csv*. Столбец text в нём содержит текст комментария, а toxic — целевой признак.

## Знакомство с данными и предобработка

In [7]:
# загрузка библиотек
!pip install catboost
!pip install warnings
!pip install wordcloud
!pip install --upgrade Pillow

ERROR: Could not find a version that satisfies the requirement warnings (from versions: none)
ERROR: No matching distribution found for warnings


In [8]:
# импортируем библиотеки

import numpy as np
import pandas as pd
import spacy
import re

from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import PassiveAggressiveClassifier
import lightgbm as lgb
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
import warnings
warnings.filterwarnings('ignore')
from catboost import CatBoostClassifier

RANDOM_STATE = 12345

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Загружаем данные из файла в датафрейм

data = pd.read_csv('/datasets/toxic_comments.csv')

In [10]:
# Функция для вывода информации об изучаемых данных

def data_info(data):
    display(data.head(10))
    print(data.info())
    print('Количество дубликатов:', sum(data.duplicated()))

In [12]:
# Вызовем функцию data_info()

data_info(data)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB
None
Количество дубликатов: 0


In [6]:
# Рассмотрим соотношение токсичных и нетоксичных комментариев

data.toxic.value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [14]:
# Выбираем случайные 5 записей из данных, где значение в столбце 'toxic' равно 1

data[data['toxic'] == 1].sample(n=10, random_state=RANDOM_STATE).head(5)

,Unnamed: 0,text,toxic
152489,152646,"I forgot to log on with my username, I correct...",1
122942,123048,Richard is a gay dancing HOBO>HOBO>HOBO!,1
26731,26766,Stop removing my edits! \nWhy do you insist on...,1
121501,121606,Stop your bullshit. I'll remove that nonsense ...,1
92382,92474,this is a pile of bull i didnt do any of this!...,1


In [15]:
# Выбираем случайные 10 не токсичных (то есть с меткой 0) элементов из DataFrame 'data' и выводим первые 5 из них

data[data['toxic'] == 0].sample(n=10, random_state=RANDOM_STATE).head(5)

,Unnamed: 0,text,toxic
113038,113136,"""\n\nmore evidence against Big.P\n\nLook at th...",0
142852,143006,Thank you. I didn't know.,0
123720,123848,Keep the article \n\nSeems to me that reasons ...,0
18757,18776,I dont replace them with blank pages. I remove...,0
75716,75792,Make it a category under Indian cuisine\nSomeo...,0


Разметка выглядит корректной. Стало понятно, что класс 1 - это токсичные комментарии, а класс 0 - все остальные. Так, получается токсичных комментариев меньше почти в 9 раз.

### Выводы:

Предоставлен датасет текста с разметкой на токсичные и не токсичные комментарии.
* Данные содержат 159292 позиции.
* Явных дубликатов и пропусков нет.
* Разметка корректна.
* Обнаружен дисбаланс классов: токсичных комментариев меньше почти в 9 раз.

## Подготовка данных

### TF-IDF

In [16]:
# Загружаем языковую модель SpaCy
nlp = spacy.load('en_core_web_sm')


# Функция для лемматизации текста с использованием SpaCy
def lemmatize_text(text):
    doc = nlp(text)
    lemmatized_output = ' '.join([token.lemma_ for token in doc])
    return lemmatized_output

# Создаем датафрейм с предложениями для тестирования
sentence1 = "The striped bats are hanging on their feet for best"
sentence2 = "you should be ashamed of yourself went worked"
df_my = pd.DataFrame([sentence1, sentence2], columns=['text'])

# Применяем функцию лемматизации на всем датасете
df_my['lemmatized_text'] = df_my['text'].apply(lemmatize_text)

print(df_my)

                                                text  \
0  The striped bats are hanging on their feet for...   
1      you should be ashamed of yourself went worked   

                                 lemmatized_text  
0  the stripe bat be hang on their foot for good  
1      you should be ashamed of yourself go work  


In [17]:
# Напишем функцию для очистки текста от всех символов, кроме букв

def clear_text(text):
    text = re.sub(r"[^a-zA-Z']", ' ', text)
    return ' '.join(text.split())

In [18]:
# Применим функции lemmatize_text и clear_text к столбцу text, создадим новый столбец text_lem
tqdm.pandas()
data['text_lem'] = data['text'].progress_apply(lambda x: lemmatize_text(clear_text(x)))

100%|██████████| 159292/159292 [45:26<00:00, 58.42it/s]


In [19]:
sampled_data = data.sample(n=1000, random_state=RANDOM_STATE)

In [20]:
# Сохраним столбец text_lem в переменную features, целевое значение toxic сохраним в переменную target
features_m = sampled_data['text_lem'].values
target_m = sampled_data['toxic']

# Создаем обучающую, тестовую и валидационную выборки
features, features_test, target, target_test = train_test_split(
    features_m, target_m, test_size=0.2, random_state=RANDOM_STATE)

features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=RANDOM_STATE)

In [23]:
# Преобразуем множество стоп-слов в список
stopwords_list = list(stopwords)

# Векторизуем features_train с использованием TF-IDF и переданных стоп-слов в виде списка
count_tf_idf = TfidfVectorizer(stop_words=stopwords_list)
tf_idf = count_tf_idf.fit_transform(features_train)
print("Размер матрицы tf-idf:", tf_idf.shape)

Размер матрицы tf-idf: (600, 5701)


Каждый текст из features_train теперь представлен в виде числового вектора с учетом важности слов и их частоты в нем. Эта матрица может быть использована для обучения для решения задачи анализа тональности (токсичности) текстов.

In [24]:
# Векторизуем features_test с TF-IDF
tf_idf_test = count_tf_idf.transform(features_test)
print("Размер матрицы tf-idf_test:", tf_idf_test.shape)

Размер матрицы tf-idf_test: (200, 5701)


In [25]:
# Векторизуем features_valid с TF-IDF
tf_idf_valid = count_tf_idf.transform(features_valid)
print("Размер матрицы tf-idf_valid:", tf_idf_valid.shape)

Размер матрицы tf-idf_valid: (200, 5701)


## Обучение моделей

In [29]:
# Создадим пустой DataFrame scores, который будет содержать результаты оценки качества модели
scores = pd.DataFrame()

In [30]:
def fit_predict_cv_with_grid_search(model, features, target, scores, cv=None, param_grid=None,
                                    features_valid=None, target_valid=None):
    # Если param_grid не задан, устанавливаем его в пустой словарь
    if param_grid is None:
        param_grid = {}

    # Создаем объект GridSearchCV для модели с указанными параметрами
    grid_search = GridSearchCV(model, param_grid=param_grid, cv=cv, scoring='f1')
    # Обучаем модель с использованием кросс-валидации
    grid_search.fit(features, target)

    # Вычисляем F1-оценку для обучающего набора данных
    F1_score_train = round(f1_score(target, grid_search.predict(features)), 2)

    # Если заданы данные для проверки, вычисляем F1-оценку для них
    if features_valid is not None and target_valid is not None:
        F1_score_valid = round(f1_score(target_valid, grid_search.predict(features_valid)), 2)
    # Иначе используем лучшую F1-оценку из кросс-валидации
    else:
        F1_score_valid = round(grid_search.best_score_, 2)

    # Добавляем результаты в список оценок
    scores = scores.append({'model': type(model).__name__,
                            'F1_score_train': F1_score_train,
                            'F1_score_valid': F1_score_valid,
                            'F1_score_train_cv': None}, ignore_index=True)

    return scores

### LGBMClassifier

In [ ]:
# Определим словарь с возможными значениями гиперпараметров для перебора
param_grid = {
    'boosting_type': ['gbdt', 'dart', 'goss'],
    'num_leaves': [20, 31, 40],
    'learning_rate': [0.1, 0.5, 1.0],
    'n_estimators': [10, 20, 50],
    'metric': ['binary_logloss', 'binary_error']
}

# Создадим модель LGBMClassifier с базовыми параметрами
model = lgb.LGBMClassifier(random_state=RANDOM_STATE)

# Вызовем функцию fit_predict_cv_with_grid_search с передачей параметров для перебора
scores = fit_predict_cv_with_grid_search(model, tf_idf, target_train, scores=scores,
                                         features_valid=tf_idf_valid, target_valid=target_valid,
                                         cv=5, param_grid=param_grid)

scores

In [32]:
scores

,model,F1_score_train,F1_score_valid,F1_score_train_cv
0,LGBMClassifier,0.71,0.06,None


### LogisticRegression

In [33]:
# Создание сетки параметров для модели Logistic Regression
param_grid = {
    'C': [0.01, 0.1, 1.0, 10.0],  # Регуляризационный параметр C
    'penalty': ['l1', 'l2'],       # Тип регуляризации: l1 - Lasso, l2 - Ridge
    'solver': ['liblinear']        # Способ оптимизации (для малых датасетов liblinear рекомендуется)
}

# Создание объекта Logistic Regression модели с балансировкой классов
model_LR = LogisticRegression(class_weight='balanced')

# Обучение модели с кросс-валидацией и поиском по сетке параметров
scores = fit_predict_cv_with_grid_search(model_LR, tf_idf, target_train, scores=scores,
                                         features_valid=tf_idf_valid, target_valid=target_valid,
                                         cv=5, param_grid=param_grid)

# Возврат результатов оценки модели
scores

,model,F1_score_train,F1_score_valid,F1_score_train_cv
0,LGBMClassifier,0.71,0.06,None
1,LogisticRegression,0.95,0.47,None


### Catboost

In [ ]:
# Задаём сетку параметров для настройки модели
param_grid = {
    'iterations': [10, 20, 50],  # Количество итераций обучения модели
    'learning_rate': [0.1, 0.5, 1.0],  # Скорость обучения модели
    'depth': [4, 6, 8]  # Глубина деревьев в градиентном бустинге
}

# Инициализируем модель CatBoostClassifier с указанными настройками
model = CatBoostClassifier(eval_metric='F1', verbose=40, random_state=RANDOM_STATE)

# Обучаем модель с кросс-валидацией и подбором параметров из заданной сетки
scores = fit_predict_cv_with_grid_search(model, tf_idf, target_train, scores=scores,
                                         features_valid=tf_idf_valid, target_valid=target_valid,
                                         cv=5, param_grid=param_grid)

# Получаем оценки модели
scores

In [35]:
scores

,model,F1_score_train,F1_score_valid,F1_score_train_cv
0,LGBMClassifier,0.71,0.06,None
1,LogisticRegression,0.95,0.47,None
2,CatBoostClassifier,0.88,0.35,None


### PassiveAggressiveClassifier

In [36]:
# Определение сетки параметров для настройки модели Passive Aggressive Classifier (PAC).
param_grid = {
    'C': [0.1, 1.0, 20.0],  # Параметр C, контролирует силу регуляризации. Большие значения C уменьшают силу регуляризации.
    'fit_intercept': [True, False],  # Определяет, следует ли оценивать перехват (intercept) в модели.
    'loss': ['hinge', 'squared_hinge']  # Функция потерь, которая определяет способ оценки ошибки модели.
}

# Инициализация модели Passive Aggressive Classifier с использованием заданного random_state.
model_PAC = PassiveAggressiveClassifier(random_state=RANDOM_STATE)

# Обучение модели PAC с кросс-валидацией и настройкой параметров сетки.
scores = fit_predict_cv_with_grid_search(model_PAC, tf_idf, target_train, scores=scores,
                                         features_valid=tf_idf_valid, target_valid=target_valid,
                                         cv=5, param_grid=param_grid)

# Сохранение лучшей модели PAC, которая была получена после настройки параметров.
best_pac_model = scores

# Получаем оценки модели
scores

,model,F1_score_train,F1_score_valid,F1_score_train_cv
0,LGBMClassifier,0.71,0.06,None
1,LogisticRegression,0.95,0.47,None
2,CatBoostClassifier,0.88,0.35,None
3,PassiveAggressiveClassifier,1.00,0.55,None


### Выводы

Были обучены модели LogisticRegression, CatBoostClassifier, LGBMClassifier и PassiveAggressiveClassifier. С моделью PassiveAggressiveClassifier удалось достичь наилучшего значения метрики F1 равной на валидационной выборке для набора данных из 1000 случайных примеров.

## Тестирование

In [ ]:
# Обучаем модель Passive Aggressive Classifier (PAC) на TF-IDF представлении данных обучения
model_PAC.fit(tf_idf, target_train)

# Делаем прогнозы на тестовых данных с использованием обученной модели
predictions_test = model_PAC.predict(tf_idf_test)

# Вычисляем F1 меру на тестовой выборке для оценки качества модели
f1_test_score = f1_score(target_test, predictions_test)

# Округляем значение F1 меры до двух знаков после запятой для удобства отображения
f1_test_score_rounded = round(f1_test_score, 2)

# Выводим значение F1 меры на тестовых данных
print('F1 мера для тестовых данных:', f1_test_score_rounded)

F1 мера для тестовых данных: 0.59


### Выводы

На тестовой выборке модель PassiveAggressiveClassifier показала результат 0.55.

## Выводы

Предоставлен датасет текста с разметкой на токсичные и не токсичные комментарии.

   * Данные содержат 159292 позиции. Явных дубликатов и пропусков нет. Столбец 'Unnamed: 0' не несет смыла и был удален.
   * Разметка корректна. Обнаружен дисбаланс классов: токсичных комментариев меньше почти в 9 раз.
   
   * Проведена предобработка данных, включая лемматизацию и очистку текста. Затем текстовые данные были векторизованы с использованием метода TF-IDF, что позволило представить тексты в виде числовых векторов, готовых для обучения модели. Теперь tf_idf содержит числовые представления обучающей выборки, tf_idf_test - для тестовой выборки, а tf_idf_valid - для валидационной выборки.
   
   * Были обучены модели LogisticRegression, CatBoostClassifier, LGBMClassifier и PassiveAggressiveClassifier. С моделью PassiveAggressiveClassifier удалось достичь наилучшего значения метрики F1 равной на валидационной выборке для набора данных из 1000 случайных примеров.
   
   * На тестовой выборке модель PassiveAggressiveClassifier показала результат 0.55.  
   